# How to Access Files for Specific Case IDs
---
This notebook demonstrates how to build a cohort of MIDRC patients based on clinical and demographic data and then obtain a file download manifest for x-ray and annotation files related to that cohort.

by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

August 2023





## 1) Set up Python environment
---


### Set local variables
---
Change the following directory paths to a valid working directories where you're running this notebook.

In [5]:
cred = "/Users/christopher/Downloads/midrc-credentials.json" # location of your MIDRC credentials, downloaded from https://data.midrc.org/identity by clicking "Create API key" button and saving the credentials.json locally
api = "https://data.midrc.org" # The base URL of the data commons being queried. This shouldn't change for MIDRC.


### Install / Import Python Packages and Scripts

In [6]:
## The packages below may be necessary for users to install according to the imports necessary in the subsequent cells.

#!pip install --upgrade pandas
#!pip install --upgrade --ignore-installed PyYAML
#!pip install --upgrade pip
#!pip install --upgrade gen3


In [7]:
## Import Python Packages and scripts

import os
import gen3

from gen3.auth import Gen3Auth
from gen3.query import Gen3Query


### Initiate instances of the Gen3 SDK Classes using credentials file for authentication
---
Again, make sure the "cred" directory path variable reflects the location of your credentials file (path variables set above).

In [8]:
auth = Gen3Auth(api, refresh_file=cred) # authentication class
query = Gen3Query(auth) # query class


## 2) Build a cohort of cases by running queries against MIDRC APIs
---
* There are many ways to query and access metadata for cohort building in MIDRC, but this notebook will focus on using the [Gen3](https://gen3.org) graphQL query service ["guppy"](https://github.com/uc-cdis/guppy/#readme). This is the backend query service that [MIDRC's data explorer GUI](https://data.midrc.org/explorer) uses. So, anything you can do in the explorer GUI, you can do with guppy queries, and more!
* The guppy graphQL service has more functionality than is demonstrated in this simple example with extensive documentation in GitHub [here](https://github.com/uc-cdis/guppy/blob/master/doc/queries.md) in case you'd like to build your own queries from scratch.
* The Gen3 SDK (intialized as "query" above in this notebook) has Python wrapper scripts to make sending queries to the guppy graphQL API simpler. The guppy SDK package can be viewed in GitHub [here](https://github.com/uc-cdis/gen3sdk-python/blob/master/gen3/query.py).


### Set 'case' query parameters
---
* Below, we first set some query parameters. Feel free to modify these parameters to see how it changes the query response. Setting these patient attributes is akin to selecting a filter value in [MIDRC's data explorer GUI](https://data.midrc.org/explorer). 
* To see more documentation about to use and combine filters with various operator logic (like AND/OR/IN, etc.) see [this page](https://github.com/uc-cdis/guppy/blob/master/doc/queries.md#filter).
* We then send our query to MIDRC's guppy API endpoint using [the Gen3Query SDK package](https://github.com/uc-cdis/gen3sdk-python/blob/master/gen3/query.py) we initialized earlier. 
* If our query request is successful, the API response should be in JSON format, and it should contain a list of patient IDs along with any other patient data we ask for.

In [9]:
#### "case" query parameters
## In this example, we're going to filter our patient cohort by asking for Asian male patients between the age of 40 and 89 that tested positive for COVID-19.

## case demographic filters
sex = "Male"
min_age = 50
max_age = 89

#### "nested" filters, these are attributes from other nodes that are nested under the case node ("child nodes" of case in the data model: data.midrc.org/dd)
## medications (vaccine data)
medication_manufacturer = ["Pfizer","Moderna"] #,"Janssen","AstraZeneca","Sinopharm","Novavax"]

## measurements filters (COVID-19 test data)
test_method = ["RT-PCR"] #,"Rapid antigen test"]
test_result_text = ["Positive","Negative"]

## conditions filters (co-morbidities and long COVID)
condition_name = ["COVID-19","Post COVID-19 condition, unspecified"] #,"Pneumonia, organism unspecified"]

## procedures filters
procedure_name = ["Breathing Support"]

In [10]:
## Here is an example getting all the cases in a particular project between ages of 45 and 47
## the "fields" option defines what fields we want the query to return. If set to "None", returns all available fields.

cases = query.raw_data_download(
                    data_type="case",
#                    fields=["project_id","submitter_id"],
                    fields=None,
                    filter_object={
                        "AND": [
                            {"=": {"sex": sex}},
                            {">=": {"age_at_index": min_age}},
                            {"<=": {"age_at_index": max_age}},
                            {"nested": {"path": "medications", "IN": {"medication_manufacturer": medication_manufacturer}}},
                            {"nested": {"path": "measurements", "IN": {"test_method": test_method}}},
                            {"nested": {"path": "measurements", "IN": {"test_result_text": test_result_text}}},
                            {"nested": {"path": "conditions", "IN": {"condition_name": condition_name}}},
                            #{"nested": {"path": "procedures", "IN": {"procedure_name": procedure_name}}}, # adding too many filters returns no data
                        ]
                    },
                    sort_fields=[{"submitter_id": "asc"}]
                )

if len(cases) > 0 and "submitter_id" in cases[0]:
    case_ids = [i['submitter_id'] for i in cases] ## make a list of the case (patient) IDs returned
    print("Query returned {} case IDs.".format(len(cases)))
    print("Data is a list with rows like this:\n\t {}".format(cases[0:1]))
else:
    print("Your query returned no data! Please, check that query parameters are valid.")

Query returned 16 case IDs.
Data is a list with rows like this:
	 [{'_case_id': '5914bdac-4d67-4bee-956d-2676b1fd1841', 'project_id': 'Open-R1', 'submitter_id': '574856-000871', 'sex': 'Male', 'race': 'White', 'age_at_index': 80, 'index_event': 'First COVID test', 'zip': '322', 'covid19_positive': 'Yes', 'ethnicity': 'Not Hispanic or Latino', 'dataset_submitter_id': ['RSNA_20230727'], 'imaging_study': 1, 'object_id': ['dg.MD1R/919469f0-a873-4194-8506-7e71a3f7665f'], 'data_format': ['DCM'], 'data_type': ['DICOM'], 'data_category': ['DX'], 'medications': [{'days_to_medication_start': 361, 'dose_sequence_number': 3, 'medication_code': '91300', 'medication_code_system': 'CPT', 'medication_manufacturer': 'Pfizer', 'medication_name': 'COVID-19 Vaccine', 'medication_type': 'Vaccine', '_medication_id': '869fcc27-c8c9-4507-833e-1c8d365b14cb'}, {'days_to_medication_start': 152, 'dose_sequence_number': 2, 'medication_code': '91300', 'medication_code_system': 'CPT', 'medication_manufacturer': 'Pfi

In [11]:
## Look at one record returned by the query
# Note: the "object_id" field is a list of all file identifiers associated with the case
cases[0]

{'_case_id': '5914bdac-4d67-4bee-956d-2676b1fd1841',
 'project_id': 'Open-R1',
 'submitter_id': '574856-000871',
 'sex': 'Male',
 'race': 'White',
 'age_at_index': 80,
 'index_event': 'First COVID test',
 'zip': '322',
 'covid19_positive': 'Yes',
 'ethnicity': 'Not Hispanic or Latino',
 'dataset_submitter_id': ['RSNA_20230727'],
 'imaging_study': 1,
 'object_id': ['dg.MD1R/919469f0-a873-4194-8506-7e71a3f7665f'],
 'data_format': ['DCM'],
 'data_type': ['DICOM'],
 'data_category': ['DX'],
 'medications': [{'days_to_medication_start': 361,
   'dose_sequence_number': 3,
   'medication_code': '91300',
   'medication_code_system': 'CPT',
   'medication_manufacturer': 'Pfizer',
   'medication_name': 'COVID-19 Vaccine',
   'medication_type': 'Vaccine',
   '_medication_id': '869fcc27-c8c9-4507-833e-1c8d365b14cb'},
  {'days_to_medication_start': 152,
   'dose_sequence_number': 2,
   'medication_code': '91300',
   'medication_code_system': 'CPT',
   'medication_manufacturer': 'Pfizer',
   'medica

## 3) Send another query to get data file details for our cohort / case ID
---
The object_id field in each case record above contains the file identifiers for all files associated with each case. If we simply want to access all files associated with our list of cases, we can use those object_ids. However, in this example, we'll ask for specific types of files and get more detailed information about each of the files. This is achieved by querying the "data_file" index and adding our cohort (list of case_ids) as a filter. 

* Note: all MIDRC data files, including both images and annotations, are listed in the guppy index "data_file", which is queried in a similar manner to our query of the "case" index above. The query parameter "data_type" below determines which Elasticsearch index we're querying.

### Set 'data_file' query parameters
---
Here, we'll utilize the property "source_node" to filter the list of files for our cohort to only those matching the type of files we're interested in. In this example, we ask for CR and DX images and any associated annotation files.

* Note: We're using the property "case_ids" as a filter to restrict the data_file records returned down to those associated with cases in our cohort built above. If you'd like to search for only one specific case_id, you can manually set the case_ids variable like this:
```
case_ids = ["my_case_id"]
```
* Or alternatively, you could set the query filter like this:
```
{"=": {"case_ids": "my_case_id"}},
```
where "my_case_id" is the quoted submitter_id of the case you're searching for.

In [12]:
source_nodes = ["cr_series_file","dx_series_file","annotation_file","dicom_annotation_file"]
modality = ["SEG", "CR", "DX", ] # this is somewhat redundant with the above source_node filter, but added here for demonstration purposes

In [13]:
## Search for specific files associated with our cohort by adding "case_ids" as a filter
# * Note: "fields" is set to "None" in this query, which by default returns all the properties available
data_files = query.raw_data_download(
                    data_type="data_file",
                    fields=None,
                    filter_object={
                        "AND": [
                            {"IN": {"case_ids": case_ids}},
                            {"IN": {"source_node": source_nodes}},
                            {"IN": {"modality": modality}},
                        ]
                    },
                    sort_fields=[{"submitter_id": "asc"}]
                )

if len(data_files) > 0:
    object_ids = [i['object_id'] for i in data_files if 'object_id' in i] ## make a list of the file object_ids returned by our query
    print("Query returned {} data files with {} object_ids.".format(len(data_files),len(object_ids)))
    print("Data is a list with rows like this:\n\t {}".format(data_files[0:1]))
else:
    print("Your query returned no data! Please, check that query parameters are valid.")

Query returned 45 data files with 45 object_ids.
Data is a list with rows like this:
	 [{'_data_file_id': '3d87c88e-12f0-4d92-ad88-4f1d9fa296b3', 'project_id': 'Open-R1', 'submitter_id': '1.2.826.0.1.3680043.10.474.574856.68485', 'series_uid': '1.2.826.0.1.3680043.10.474.574856.68485', 'case_ids': ['574856-000667'], 'object_id': 'dg.MD1R/40a900c7-a0d0-45b8-8dc4-c1714d9a5e88', 'md5sum': '075d7d467769392c1b25e392c1c39bf2', 'file_name': '574856-000667/1.2.826.0.1.3680043.10.474.574856.68484/1.2.826.0.1.3680043.10.474.574856.68485.zip', 'file_size': 9692948, 'data_format': 'DCM', 'data_type': 'DICOM', 'data_category': 'DX', 'lossy_image_compression': '00', 'manufacturer': 'KONICA MINOLTA', 'manufacturer_model_name': 'CS-7', 'modality': 'DX', 'series_description': 'AP', 'source_node': 'dx_series_file', 'detector_type': 'SCINTILLATOR', 'image_type': ['ORIGINAL_PRIMARY'], 'imager_pixel_spacing': [0.175], 'pixel_spacing': [0.175], 'program_name': ['Open'], 'project_code': ['R1'], '_dataset_id'

In [14]:
## View the detailed data for the first file returned
data_files[0]

{'_data_file_id': '3d87c88e-12f0-4d92-ad88-4f1d9fa296b3',
 'project_id': 'Open-R1',
 'submitter_id': '1.2.826.0.1.3680043.10.474.574856.68485',
 'series_uid': '1.2.826.0.1.3680043.10.474.574856.68485',
 'case_ids': ['574856-000667'],
 'object_id': 'dg.MD1R/40a900c7-a0d0-45b8-8dc4-c1714d9a5e88',
 'md5sum': '075d7d467769392c1b25e392c1c39bf2',
 'file_name': '574856-000667/1.2.826.0.1.3680043.10.474.574856.68484/1.2.826.0.1.3680043.10.474.574856.68485.zip',
 'file_size': 9692948,
 'data_format': 'DCM',
 'data_type': 'DICOM',
 'data_category': 'DX',
 'lossy_image_compression': '00',
 'manufacturer': 'KONICA MINOLTA',
 'manufacturer_model_name': 'CS-7',
 'modality': 'DX',
 'series_description': 'AP',
 'source_node': 'dx_series_file',
 'detector_type': 'SCINTILLATOR',
 'image_type': ['ORIGINAL_PRIMARY'],
 'imager_pixel_spacing': [0.175],
 'pixel_spacing': [0.175],
 'program_name': ['Open'],
 'project_code': ['R1'],
 '_dataset_id': ['a6c328b2-505e-4098-b641-eae914795163'],
 '_case_id': ['d7ce0

## 4) Access data files using their object_id / data GUID (globally unique identifiers)
---
In order to download files stored in MIDRC, users need to reference the file's object_id (AKA data GUID or Globally Unique IDentifier).

Once we have a list of GUIDs we want to download, we can use either the gen3-client or the gen3 SDK to download the files. You can also access individual files in your browser after logging-in and entering the GUID after the `files/` endpoint, as in this URL: https://data.midrc.org/files/GUID

where GUID is the actual GUID, e.g.: https://data.midrc.org/files/dg.MD1R/b87d0db3-d95a-43c7-ace1-ab2c130e04ec

For instructions on how to install and use the gen3-client, please see [the MIDRC quick-start guide](https://data.midrc.org/dashboard/Public/documentation/Gen3_MIDRC_GetStarted.pdf), which can be found linked here and in the MIDRC data portal header as "Get Started".

Below we use the gen3 SDK command `gen3 drs-pull object` which is [documented in detail here](https://github.com/uc-cdis/gen3sdk-python/blob/master/docs/howto/drsDownloading.md).

### Parse the data_file query response to build a list of all `object_id`s returned for our cohort. 

In [15]:
## Build a list 
object_ids = []
for data_file in data_files:
    if 'object_id' in data_file:
        object_id = data_file['object_id']
        object_ids.append(object_id)

object_id = object_ids[1]
print("The first object_id of {}: '{}'".format(len(object_ids),object_id))

The first object_id of 45: 'dg.MD1R/f3d90ecf-c383-4a1d-8af9-c1bd8f1cb051'


### Use the Gen3 SDK command `gen3 drs-pull object` to download an individual file

In [16]:
## Make a new directory for downloaded files
os.system("mkdir -p downloads")


0

In [17]:
## Run the "gen3 drs-pull object" command to download a file
cmd = "gen3 --auth {} --endpoint data.midrc.org drs-pull object {} --output-dir downloads".format(cred,object_id)
os.system(cmd)


{"succeeded": ["dg.MD1R/f3d90ecf-c383-4a1d-8af9-c1bd8f1cb051"], "failed": []}


0

In [18]:
!find downloads -name "*dcm"

downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.272228/1.2.826.0.1.3680043.10.474.574856.272229/1.2.826.0.1.3680043.10.474.574856.272230.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.272228/1.2.826.0.1.3680043.10.474.574856.272229/1.2.826.0.1.3680043.10.474.574856.272227.dcm
downloads/574856-004380/1.2.826.0.1.3680043.10.474.574856.2166015/1.2.826.0.1.3680043.10.474.574856.2166016/1.2.826.0.1.3680043.10.474.574856.2166014.dcm


### Use a simple loop to download all the files

In [19]:
## Simple loop to download all files and keep track of success and failures
cred = "/Users/christopher/Downloads/midrc-credentials.json" # location of your MIDRC credentials, downloaded from https://data.midrc.org/identity by clicking "Create API key" button and saving the credentials.json locally
success,failure,other=[],[],[]
count,total = 0,len(object_ids)
for object_id in object_ids:
    count+=1
    cmd = "gen3 --auth {} --endpoint data.midrc.org drs-pull object {} --output-dir downloads".format(cred,object_id)
    stout = subprocess.run(cmd, shell=True, capture_output=True)
    print("Progress ({}/{}): {}".format(count,total,stout.stdout))
    if "failed" in str(stout.stdout):
        failure.append(object_id)
    elif "successfully" in str(stout.stdout):
        success.append(object_id)
    else:
        other.append(object_id)

Progress (1/45): b'{"succeeded": ["dg.MD1R/40a900c7-a0d0-45b8-8dc4-c1714d9a5e88"], "failed": []}\n'
Progress (2/45): b'{"succeeded": ["dg.MD1R/f3d90ecf-c383-4a1d-8af9-c1bd8f1cb051"], "failed": []}\n'
Progress (3/45): b'{"succeeded": ["dg.MD1R/5abec3d4-e828-4ec3-a6aa-19876fcdfd16"], "failed": []}\n'
Progress (4/45): b'{"succeeded": ["dg.MD1R/b7550986-ca83-43bd-bdf7-59180a22756c"], "failed": []}\n'
Progress (5/45): b'{"succeeded": ["dg.MD1R/b2066d91-bd09-4334-9987-23a723561f0a"], "failed": []}\n'
Progress (6/45): b'{"succeeded": ["dg.MD1R/19a00d00-e9b8-4ec3-b8aa-7c83098e8b30"], "failed": []}\n'
Progress (7/45): b'{"succeeded": ["dg.MD1R/cdc4b1f6-369d-48e9-8182-483dd228642a"], "failed": []}\n'
Progress (8/45): b'{"succeeded": ["dg.MD1R/015e8d04-dcee-461f-9df0-526c735d0fc8"], "failed": []}\n'
Progress (9/45): b'{"succeeded": ["dg.MD1R/0d550de6-5cee-43ef-9ba5-b4c4e176ee76"], "failed": []}\n'
Progress (10/45): b'{"succeeded": ["dg.MD1R/17034b8a-738e-4800-a447-d5594a55561d"], "failed": []}\n'

In [20]:
!find downloads -name "*.dcm"

downloads/574856-000491/1.2.826.0.1.3680043.10.474.574856.38401/1.2.826.0.1.3680043.10.474.574856.38402/1.2.826.0.1.3680043.10.474.574856.38400.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.56108/1.2.826.0.1.3680043.10.474.574856.56109/1.2.826.0.1.3680043.10.474.574856.56107.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.284869/1.2.826.0.1.3680043.10.474.574856.284870/1.2.826.0.1.3680043.10.474.574856.284868.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.252187/1.2.826.0.1.3680043.10.474.574856.252188/1.2.826.0.1.3680043.10.474.574856.252186.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.164169/1.2.826.0.1.3680043.10.474.574856.164170/1.2.826.0.1.3680043.10.474.574856.164168.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.272228/1.2.826.0.1.3680043.10.474.574856.272229/1.2.826.0.1.3680043.10.474.574856.272230.dcm
downloads/574856-000667/1.2.826.0.1.3680043.10.474.574856.272228/1.2.826.0.1.3680043.10.474.574856.2

In [21]:
!find downloads -name "*.dcm" | wc -l

      55


## The End
---
If you have any questions related to this notebook don't hesitate to reach out to the MIDRC Helpdesk at midrc-support@datacommons.io or the author directly at cgmeyer@uchicago.edu

Happy data wrangling!